In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy import stats
import warnings
pd.options.mode.chained_assignment = None 



In [2]:
vdf = pd.read_csv("beeActivity.csv")
vdf['track_endtime'] = vdf['track_endtime'].apply(lambda x: pd.to_datetime(x))
vdf['track_starttime'] = vdf['track_starttime'].apply(lambda x: pd.to_datetime(x))
vdf['track_tagid'] = vdf['track_tagid'].apply(lambda x: str(x))

In [3]:
#SECONDS THRESHOLD
#used to classify multiple detections into part of a single event
#also used to tell when two detections are part of different events
#by checking the time distance between them
t = 15

#DISTANCE THRESHOLD
#used to classify an event as entering or exiting
#when two consecutive detections of the same event 
#have this distance in y position, they are utilized to predict
#the trajectory. If not then it checks the detection prior for
#the distance threshold, and continues doing so until it finds
#the last detection in the event or until it finds a distance
#of more than the threshold

t2 = 150

#ANGLE THRESHOLD
#used to generate angle ranges for classifying as exiting
#or entering

angle = 10

## Look at events prior and classify based on y displacement

In [4]:
def beeCleanPrior(bee):

    id = bee['track_tagid'].iloc[0]

    new_event = []
    datetime = []
        
    for i in range(len(bee)-1):

            time = bee['track_endtime'].iloc[i]
            next_t = bee['track_starttime'].iloc[i+1]

            init = bee['track_starty'].iloc[i]

            #if time passed is greater than the threshold,
            #classify the last detection in an event
            if (next_t - time).total_seconds() > t:
                second = bee['track_endy'].iloc[i]
                matched = False
                counter = 1
                #iterate backwards until the distance threshold
                #is met or until they reach the first detection in
                #the event
                while not matched: 
                    if abs(second-init) > t2:
                        if second > init:
                            new_event.append('exiting')
                        else:
                            new_event.append('entering')
                        matched = True
                    elif (time - bee['track_starttime'].iloc[i-counter]).total_seconds() < t:
                        init = bee['track_starty'].iloc[i-counter]
                        counter += 1
                    else:
                        if second > init:
                            new_event.append('exiting')
                        elif init > second:
                            new_event.append('entering')
                        else:
                            new_event.append('unknown')
                        matched = True
                datetime.append(time)
                
                
            
    tagID = [id] * len(new_event)
    df = pd.DataFrame.from_dict({'tagID': tagID, 'datetime':datetime, 'event':new_event})
    return df

In [5]:
bees = []

beeIDs = vdf['track_tagid'].unique()
for bee in beeIDs:

    b = vdf[vdf['track_tagid'] == bee].copy().reset_index()
    events = beeCleanPrior(b)
    bees.append(events)
    
    
new = pd.concat(bees, axis = 0).reset_index() 
new = new[['tagID','datetime','event']]
new.to_csv("bee_prior.csv", index=False)
new
prior = new

C:\Users\kashi\AppData\Local\Temp\ipykernel_6404\1337217809.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new = pd.concat(bees, axis = 0).reset_index()


## Threshold-based approach

In [6]:
entering = ["outside_inside","ramp_inside","inside_inside"]
exiting = ["inside_outside","ramp_outside","outside_outside"]

def beeCleanPrior(bee):

    id = bee['track_tagid'].iloc[0]

    new_event = []
    datetime = []
        
    for i in range(len(bee)-1):

            time = bee['track_endtime'].iloc[i]
            next_t = bee['track_starttime'].iloc[i+1]

            #if time passed is greater than the threshold,
            #classify the last detection in an event
            if (next_t - time).total_seconds() > t:
                if bee['track_shape'].iloc[i] in entering:
                    new_event.append('entering')
                elif bee['track_shape'].iloc[i] in exiting:
                    new_event.append('exiting')
                else:
                    new_event.append('unknown')
                datetime.append(time)
                
                
            
    tagID = [id] * len(new_event)
    df = pd.DataFrame.from_dict({'tagID': tagID, 'datetime':datetime, 'event':new_event})
    return df

In [7]:
bees = []

beeIDs = vdf['track_tagid'].unique()
for bee in beeIDs:

    b = vdf[vdf['track_tagid'] == bee].copy().reset_index()
    events = beeCleanPrior(b)
    bees.append(events)
    
    
new = pd.concat(bees, axis = 0).reset_index() 
new = new[['tagID','datetime','event']]
new.to_csv("bee_threshold.csv", index=False)
new
threshold = new

C:\Users\kashi\AppData\Local\Temp\ipykernel_6404\1632906819.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new = pd.concat(bees, axis = 0).reset_index()


## Classify events based on summed vector angle of all detections corresponding to an event

Obtained from BeeCam-AprilTag
https://github.com/AERS-Lab/BeeCam-AprilTag

In [8]:
def beeCleanAngle(bee):

    id = bee['track_tagid'].iloc[0]
    new_event = []
    datetime = []
    vector_x = []
    vector_y = []

    
    enter_min = 180 + angle
    enter_max = 360 - angle
    exit_min = angle
    exit_max = 180 - angle
        
    
    for i in range(len(bee)-1):

            time = bee['track_endtime'].iloc[i]
            next_t = bee['track_starttime'].iloc[i+1]

            #obtain direction vector 
            #normalized to a magnitude of 1
            y1 = bee['track_starty'].iloc[i]
            y2 = bee['track_endy'].iloc[i]
            x1 = bee['track_startx'].iloc[i]
            x2 = bee['track_endx'].iloc[i]

            dx = x2-x1
            dy = y2-y1
            angle_rad = np.arctan2(dy, dx)
            unit_dx = np.cos(angle_rad)
            unit_dy = np.sin(angle_rad)

            vector_x.append(unit_dx)
            vector_y.append(unit_dy)
            
            #if next detection is further than the threshold,
            #utilize all stored angles to calculate event trajectory
            if (next_t - time).total_seconds() > t:
                #obtain average
                avg_x = sum(vector_x)/len(vector_x)
                avg_y = sum(vector_y)/len(vector_y)

                if avg_x == 0 and avg_y == 0:
                    deg = 0
                elif avg_x == 0 and avg_y != 0:
                    if avg_y > 0:
                        deg = 270
                    elif avg_y < 0:
                        deg = 90
                else:
                    # determine direction angle using arctan
                    deg = np.rad2deg(np.arctan(avg_y/avg_x))
                    
                    # since arctan limits are (-90,90), use coordinate directions to 
                    # correct the angle to be within standard [0,360) range
                    if avg_x > 0 and avg_y >= 0:
                        deg = deg
                    elif avg_x < 0 and avg_y >= 0:
                        deg = 180 + deg
                    elif avg_x < 0 and avg_y < 0:
                        deg = deg + 180
                    elif avg_x > 0 and avg_y < 0:
                        deg = 360 + deg

                #classify events based on angle threshold
                if deg >= exit_min and deg <= exit_max:
                    new_event.append('exiting')
                elif deg >= enter_min and deg <= enter_max:
                    new_event.append('entering')
                else:
                    new_event.append('unknown')
                datetime.append(time)
                vector_x = []
                vector_y = []
            
    tagID = [id] * len(new_event)
    df = pd.DataFrame.from_dict({'tagID': tagID, 'datetime':datetime, 'event':new_event})
    #print(bee)
    return df

In [9]:
bees = []

beeIDs = vdf['track_tagid'].unique()
for bee in beeIDs:

    b = vdf[vdf['track_tagid'] == bee].copy().reset_index()
    events = beeCleanAngle(b)
    #print(events)
    bees.append(events)
    
    
new = pd.concat(bees, axis = 0).reset_index() 
new = new[['tagID','datetime','event']]
new.to_csv("bee_angle.csv", index=False)
new
summed = new

C:\Users\kashi\AppData\Local\Temp\ipykernel_6404\1310600663.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new = pd.concat(bees, axis = 0).reset_index()


## Classify events based on last angle corresponding to an event

Modified from BeeCam-AprilTag
https://github.com/AERS-Lab/BeeCam-AprilTag

In [10]:
def beeCleanSingleAngle(bee):

    id = bee['track_tagid'].iloc[0]
    new_event = []
    datetime = []
    
    enter_min = 180 + angle
    enter_max = 360 - angle
    exit_min = angle
    exit_max = 180 - angle
        
    
    for i in range(len(bee)-1):

            time = bee['track_endtime'].iloc[i]
            next_t = bee['track_starttime'].iloc[i+1]

            #if time is over threshold
            #utilize last movement angle to predict trajectory
            if (next_t - time).total_seconds() > t:

                y1 = bee['track_starty'].iloc[i]
                y2 = bee['track_endy'].iloc[i]
                x1 = bee['track_startx'].iloc[i]
                x2 = bee['track_endx'].iloc[i]
    
                dx = x2-x1
                dy = y2-y1
                angle_rad = np.arctan2(dy, dx)
                avg_x = np.cos(angle_rad)
                avg_y = np.sin(angle_rad)

                if avg_x == 0 and avg_y == 0:
                    deg = 0
                elif avg_x == 0 and avg_y != 0:
                    if avg_y > 0:
                        deg = 270
                    elif avg_y < 0:
                        deg = 90
                else:
                    # determine direction angle using arctan
                    deg = np.rad2deg(np.arctan(avg_y/avg_x))
                    
                    # since arctan limits are (-90,90), use coordinate directions to 
                    # correct the angle to be within standard [0,360) range
                    if avg_x > 0 and avg_y >= 0:
                        deg = deg
                    elif avg_x < 0 and avg_y >= 0:
                        deg = 180 + deg
                    elif avg_x < 0 and avg_y < 0:
                        deg = deg + 180
                    elif avg_x > 0 and avg_y < 0:
                        deg = 360 + deg

                #print(deg)
                if deg >= exit_min and deg <= exit_max:
                    new_event.append('exiting')
                elif deg >= enter_min and deg <= enter_max:
                    new_event.append('entering')
                else:
                    new_event.append('unknown')
                datetime.append(time)
            
    tagID = [id] * len(new_event)
    df = pd.DataFrame.from_dict({'tagID': tagID, 'datetime':datetime, 'event':new_event})
    #print(bee)
    return df

In [11]:
bees = []

beeIDs = vdf['track_tagid'].unique()
for bee in beeIDs:

    b = vdf[vdf['track_tagid'] == bee].copy().reset_index()
    events = beeCleanSingleAngle(b)
    #print(events)
    bees.append(events)
    
    
new = pd.concat(bees, axis = 0).reset_index() 
new = new[['tagID','datetime','event']]
new.to_csv("bee_singleangle.csv", index=False)
new
single = new

C:\Users\kashi\AppData\Local\Temp\ipykernel_6404\3964587374.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new = pd.concat(bees, axis = 0).reset_index()


In [12]:
cheatsheet = pd.read_csv("cheatsheet.csv")


In [13]:
correct = {'prior':0,'summedvector':0,'single':0,'threshold':0}
for index, row in cheatsheet.iterrows():
    if prior['event'].iloc[int(row['line'])-1] == row['event']:
        correct['prior'] += 1
    if summed['event'].iloc[int(row['line'])-1] == row['event']:
        correct['summedvector'] += 1
    if single['event'].iloc[int(row['line'])-1] == row['event']:
        correct['single'] += 1
    if threshold['event'].iloc[int(row['line'])-1] == row['event']:
        correct['threshold'] += 1

results = {k: v/len(cheatsheet) for k, v in correct.items()}
results


{'prior': 0.8901098901098901,
 'summedvector': 0.1043956043956044,
 'single': 0.8296703296703297,
 'threshold': 0.45054945054945056}

In [14]:
len(prior[prior['event'] == summed['event']])/len(prior)

0.8692253020611229